# Autonomous development of turn-taking behaviors in agent populations: a computational study

This notebook shows how to run and plot the experiments. For the oldest version of the code related to [our paper pubished at the ICDL-Epirob 2015 conference](http://clement-moulin-frier.github.io/bibliography/moulinfrier2015turntaking/), use the branch `icdl_paper` instead.

First instanciate the agent population of the experiment:

In [1]:
from agent import Agent
from experiment import Experiment

# number of agents in the population
n_ag = 3

# agent parameters: 
# i is the agent id, 
# the two following parameters are the mean and variance of their specific vocalization feature 
# (ie the auditory feature provided to the adaptive layer)
ag_voc_params = []
for i in range(n_ag):
    ag_voc_params.append((i, [- n_ag / 2. + i], [[0.0001]])) 
    
# Instanciate an experiment with those agents
expe = Experiment([Agent(ag_voc_params, i) for i, _ in enumerate(ag_voc_params)])

Let's start with reactive agents by disabling their adaptive layer (Section III-B of the paper):

In [2]:
for ag in expe.ags:
    ag.adapt = False

Then run the experiment, ie the agent interactions, during a number of time steps:

In [3]:
# Run the experiment for 100 time steps
expe.run(100)

Then plot the results in figures. We use the [seaborn library](http://stanford.edu/~mwaskom/software/seaborn/) that you need to install on your system to be able to generate the figures. Figures are plotted into pdf files in the root repository directory.

In [4]:
# import figure generation classes
import sys
#sys.path.append('figure_generation')
#cd "figure_generation"
from figure_generation.figure_generation import Reactive, Adaptive, ActionPolicy, RewardStat, sns

Figure 2 of the paper regarding the reactive layer results:

In [5]:
# Set formatting figure parameters (fontsize etc...)
sns.set_context("paper", font_scale=2.3, rc={"lines.markersize": 8., "lines.linewidth": 4.})

# generate the figure in pdf from the Experiment instance:
fig_gen = Reactive(expe)
fig_gen.generate('reactive.pdf')

Now let's run an experiment with adaptive agents (Section III-C of the paper). First delete the previous one:

In [6]:
for ag in expe.ags:
    del ag
del expe

And instanciate a new experiment using the same vocalization parameters but without disabling the adaptive layer of the agents: 

In [8]:
# Instanciate an experiment with those agents
expe = Experiment([Agent(ag_voc_params, i) for i, _ in enumerate(ag_voc_params)])

Run this new experiment on a (higher) number of time steps

In [9]:
# Run the experiment for 11000 time steps
expe.run(11000)

Generate Figure 3 of the paper regarding the adaptive layer results:

In [10]:
# Set formatting figure parameters (fontsize etc...)
sns.set_context("paper", font_scale=2.3, rc={"lines.markersize": 8., "lines.linewidth": 4.})

# generate the figure in pdf from the Experiment instance:
fig_gen = Adaptive(expe)
fig_gen.time_ranges = ((0, 100), (5000, 5100), (10900, 11000))
fig_gen.generate('adaptive.pdf')

Figure 4 of the paper regarding the learned action policies:

In [ ]:
# Set formatting figure parameters (fontsize etc...)
sns.set_context("paper", font_scale=1.5, rc={"lines.markersize": 8., "lines.linewidth": 4.})

# generate the figure in pdf from the Experiment instance:
fig_gen = ActionPolicy(expe)
fig_gen.generate('policy.pdf')

Finally lets run more experiments with more agents to check whether the model performances are robust (Figure 5 of the paper).

First delete the previous experiment:

In [11]:
for ag in expe.ags:
    del ag
del expe

And create a number of identical and independent experiments (let's begin with 2-agent populations as above):

In [12]:
# Number of experiments
n_expe = 10

# Instanciations
expes = []
for i in range(n_expe):
    expes.append(Experiment([Agent(ag_voc_params, i) for i, _ in enumerate(ag_voc_params)]))

Run all of them (the $bg=True$ argument allows to run them in parallel):

In [13]:
for expe in expes:
    expe.run(11000, bg=True)

Wait for all the experiments to terminate (it can take a while, say up to 10mn):

In [14]:
for expe in expes:
    expe.wait()

Save the logs of these 2-agent experiments:

In [15]:
expe_logs = [expe.log for expe in expes]
import cPickle as pickle
pickle.dump(expe_logs, open('expe_logs_2ag.pickle', 'wb'))

And delete the experiments:

In [16]:
for expe in expes:
    for ag in expe.ags:
        del ag
    del expe
del expes

Now do the same with 3-agent populations:

In [17]:
n_ag = 3

# agent vocalization parameters
ag_voc_params = []
for i in range(n_ag):
    ag_voc_params.append((i, [- n_ag / 2. + i], [[0.0001]])) 
    
# Instanciatite the 3-agent experiments
expes = []
for i in range(n_expe):
    expes.append(Experiment([Agent(ag_voc_params, i) for i, _ in enumerate(ag_voc_params)]))

Run all of them and wait (it can take a while again, even more due to the higher number of agents and time steps):

In [18]:
for expe in expes:
    expe.run(30000, bg=True)
    
for expe in expes:
    expe.wait()

Save the logs of these 3-agent experiments:

In [19]:
expe_logs = [expe.log for expe in expes]

Finally, generate Figure 5 of the paper from the saved logs (it can take a lot of RAM, please check you have enough free memory to avoid freezing):

In [ ]:
# Set formatting figure parameters (fontsize etc...)
sns.set_context("paper", font_scale=2., rc={"lines.markersize": 8., "lines.linewidth": 4.})

# load the saved log for 2-agent experiments
import cPickle as pickle
expe_logs_2ag = pickle.load(open('expe_logs_2ag.pickle', 'rb'))

# rename the logs for 3-agent experiments
expe_logs_3ag = expe_logs

# generate the figure in pdf from the Experiment instance:
fig_gen = RewardStat(expe_logs_2ag, expe_logs_3ag)
fig_gen.generate('reward_stat.pdf')